[**Blueprints for Text Analysis Using Python**](https://github.com/blueprints-for-text-analytics-python/blueprints-text)  
Jens Albrecht, Sidharth Ramachandran, Christian Winkler

**If you like the book or the code examples here, please leave a friendly comment on [Amazon.com](https://www.amazon.com/Blueprints-Text-Analytics-Using-Python/dp/149207408X)!**
<img src="../rating.png" width="100"/>


# Chapter 7: How to Explain a Text Classifier<div class='tocSkip'/>

## Remark<div class='tocSkip'/>

The code in this notebook differs slightly from the printed book. 

Several layout and formatting commands, like `figsize` to control figure size or subplot commands are removed in the book.

All of this is done to simplify the code in the book and put the focus on the important parts instead of formatting.

## Setup<div class='tocSkip'/>

Set directory locations. If working on Google Colab: copy files and install required libraries.

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master'
    os.system(f'wget {GIT_ROOT}/ch07/setup.py')

%run -i setup.py

## Load Python Settings<div class="tocSkip"/>

Common imports, defaults for formatting in Matplotlib, Pandas etc.

In [ ]:
%run "$BASE_DIR/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# From classification chapter

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 50)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Cleaning up the data to remove special characters - will re-use the blueprint from Chapter 5
import html 
import re
# tags like 
RE_TAG = re.compile(r'<[^<>]*>')
# text or code in brackets like [0]
RE_BRACKET = re.compile('\[[^\[\]]*\]')
# text or code in brackets like (0)
RE_BRACKET_1 = re.compile('\([^)]*\)')
# specials that are not part of words; matches # but not #cool
RE_SPECIAL = re.compile(r'(?:^|\s)[&#<>{}\[\]+]+(?:\s|$)')
# standalone sequences of hyphens like --- or ==
RE_HYPHEN_SEQ = re.compile(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)')
# sequences of white spaces
RE_MULTI_SPACE = re.compile('\s+')

def clean(text):
    text = html.unescape(text)
    text = RE_TAG.sub(' ', text)
    text = RE_BRACKET.sub(' ', text)
    text = RE_BRACKET_1.sub(' ', text)
    text = RE_SPECIAL.sub(' ', text)
    text = RE_HYPHEN_SEQ.sub(' ', text)
    text = RE_MULTI_SPACE.sub(' ', text)
    return text.strip()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

# Final Blueprint for Text Classification

In [ ]:
# Loading the dataframe

df = pd.read_csv(BUGS_FILE)
df = df.groupby('Component', as_index=False).apply(pd.DataFrame.sample, random_state=42, frac=.2)
df = df[['Title','Description','Component']]
df = df.dropna()
df['text'] = df['Title'] + " " + df['Description']
df = df.drop(columns=['Title','Description'])

# Step 1 - Data Preparation

df['text'] = df['text'].apply(clean)

# Step 2 - Train-Test Split

X_train, X_test, Y_train, Y_test = train_test_split(df['text'], df['Component'], 
                                                    test_size=0.2, random_state=42,
                                                    stratify=df['Component'])
print ('Size of Training Data ', X_train.shape[0])
print ('Size of Test Data ', X_test.shape[0])

# Step 3 - Training the Machine Learning model

tfidf = TfidfVectorizer(min_df = 10, ngram_range=(1,2), stop_words="english")
X_train_tf = tfidf.fit_transform(X_train)


svc = SVC(kernel="linear", C=1, probability=True, random_state=42)
svc.fit(X_train_tf, Y_train)

In [ ]:
X_test_tf = tfidf.transform(X_test)
Y_pred = svc.predict(X_test_tf)
result = pd.DataFrame({ 'text': X_test.values, 'actual': Y_test.values, 'predicted': Y_pred })

# Explainable AI

In [ ]:
result[result["actual"] != result["predicted"]].head()

In [ ]:
text = result.iloc[21]["text"]
print(text)

In [ ]:
svc.predict_proba(X_test_tf[21])

In [ ]:
class_names = ["APT", "Core", "Debug", "Doc", "Text", "UI"]
prob = svc.predict_proba(X_test_tf)
# new dataframe for explainable results
er = result.copy().reset_index()
for i, c in enumerate(class_names):
    er[c] = prob[:, i]

In [ ]:
er[["actual", "predicted"] + class_names].sample(5, random_state=99)

In [ ]:
er['max_probability'] = er[class_names].max(axis=1)
correct = (er[er['actual'] == er['predicted']])
wrong   = (er[er['actual'] != er['predicted']])

In [ ]:
import matplotlib.pyplot as plt
correct["max_probability"].plot.hist(title="Correct")
plt.savefig("correct.svg")

In [ ]:
wrong["max_probability"].plot.hist(title="Wrong")
plt.savefig("wrong.svg")

In [ ]:
len(correct)

In [ ]:
len(wrong)

In [ ]:
high = er[er["max_probability"] > 0.8]

In [ ]:
len(high)

In [ ]:
print(classification_report(high["actual"], high["predicted"]))

In [ ]:
print(classification_report(er["actual"], er["predicted"]))

## then with most important components

In [ ]:
svc.coef_

In [ ]:
# coef_[1] yields a matrix, A[0] converts to array and takes first row
coef = svc.coef_[8].A[0]
vocabulary_positions = coef.argsort()
vocabulary = tfidf.get_feature_names_out()

In [ ]:
top_words = 10
top_positive_coef = vocabulary_positions[-top_words:].tolist()
top_negative_coef = vocabulary_positions[:top_words].tolist()

In [ ]:
core_ui = pd.DataFrame([[vocabulary[c], coef[c]] for c in top_positive_coef + top_negative_coef], 
                          columns=["feature", "coefficient"]).sort_values("coefficient")

In [ ]:
core_ui.set_index("feature")

In [ ]:
10*['r']

In [ ]:
core_ui.set_index("feature").plot.barh(color=[['red']*10 + ['green']*10])
plt.savefig("coefficients-core-ui.svg")

In [ ]:
c = svc.coef_
coef = (c[5] + c[6] + c[7] + c[8] - c[0]).A[0]
vocabulary_positions = coef.argsort()

In [ ]:
top_words = 20
top_positive_coef = vocabulary_positions[-top_words:].tolist()
top_negative_coef = vocabulary_positions[:top_words].tolist()
core = pd.DataFrame([[vocabulary[c], coef[c]] for c in top_positive_coef + top_negative_coef], 
                       columns=["feature", "coefficient"]).sort_values("coefficient")
core.set_index("feature").plot.barh(figsize=(6, 10), color=[['red']*top_words + ['green']*top_words])
plt.savefig("coefficients-core.svg")

## then with LIME

In [ ]:
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(tfidf, svc)

In [ ]:
pipeline.predict_proba(["compiler not working"])

In [ ]:
class_names

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
er[er["predicted"] != er["actual"]].head(5)

In [ ]:
id = 21
print('Document id: %d' % id)
print('Predicted class =', er.iloc[id]["predicted"])
print('True class: %s' % er.iloc[id]["actual"])

In [ ]:
exp = explainer.explain_instance(result.iloc[id]["text"], pipeline.predict_proba, num_features=10, labels=[1, 5])
print('Explanation for class %s' % class_names[1])
print('\n'.join(map(str, exp.as_list(label=1))))
print()
print('Explanation for class %s' % class_names[5])
print('\n'.join(map(str, exp.as_list(label=5))))

In [ ]:
exp = explainer.explain_instance(result.iloc[id]["text"], pipeline.predict_proba, num_features=6, top_labels=3)
print(exp.available_labels())

In [ ]:
exp.show_in_notebook(text=False)

In [ ]:
from lime import submodular_pick

In [ ]:
%%time
import numpy as np
np.random.seed(42)
lsm = submodular_pick.SubmodularPick(explainer, er["text"].values, pipeline.predict_proba, 
                                        sample_size=100,
                                        num_features=20,
                                        num_exps_desired=5)

In [ ]:
lsm.explanations[0].show_in_notebook()

# ELI5

In [ ]:
from sklearn.linear_model import SGDClassifier
svm = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
svm.fit(X_train_tf, Y_train)
Y_pred_svm = svm.predict(X_test_tf)
print(classification_report(Y_test, Y_pred_svm))

In [ ]:
import eli5
eli5.show_weights(svm, top=10, vec=tfidf, target_names=class_names)

In [ ]:
eli5.show_prediction(svm, X_test.iloc[21],  vec=tfidf, target_names=class_names)

## then with anchor

In [ ]:
import spacy
from anchor import anchor_text
import time
import numpy as np

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
# hack for spacy 2.3
for s in nlp.vocab.vectors:
    _ = nlp.vocab[s]

In [ ]:
import numpy as np
np.random.seed(42)
explainer_unk = anchor_text.AnchorText(nlp, class_names, use_unk_distribution=True)

In [ ]:
text = er.iloc[21]["text"]
actual = er.iloc[21]["actual"]
# we want the class with the highest probability and must invert the order
predicted_class_ids = np.argsort(pipeline.predict_proba([text])[0])[::-1]
pred = explainer_unk.class_names[predicted_class_ids[0]]
alternative = explainer_unk.class_names[predicted_class_ids[1]]
print(f'predicted {pred}, alternative {alternative}, actual {actual}')

In [ ]:
exp_unk = explainer_unk.explain_instance(text, pipeline.predict, threshold=0.95)

In [ ]:
print(f'Rule: {" AND ".join(exp_unk.names())}')
print(f'Precision: {exp_unk.precision()}')

In [ ]:
print(f'Made-up examples where anchor rule matches and model predicts {pred}\n')
print('\n'.join([x[0] for x in exp_unk.examples(only_same_prediction=True)]))

In [ ]:
print(f'Made-up examples where anchor rule matches and model predicts {alternative}\n')
print('\n'.join([x[0] for x in exp_unk.examples(partial_index=0, only_different_prediction=True)]))

## Without unknown

In [ ]:
np.random.seed(42)
explainer_no_unk = anchor_text.AnchorText(nlp, class_names, use_unk_distribution=False, use_bert=False)
exp_no_unk = explainer_no_unk.explain_instance(text, pipeline.predict, threshold=0.95)

In [ ]:
print(f'Rule: {" AND ".join(exp_no_unk.names())}')
print(f'Precision: {exp_no_unk.precision()}')

In [ ]:
print('Anchor: %s' % (' AND '.join(exp_no_unk.names())))
print('Precision: %.2f' % exp_no_unk.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp_no_unk.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp_no_unk.examples(partial_index=0, only_different_prediction=True)]))

## numerical for show in notebook

In [ ]:
def predict_numerical(text):
    res = pipeline.predict(text)
    n = np.array([str(class_names.index(r)) for r in res])
    return n

In [ ]:
explainer_num = anchor_text.AnchorText(nlp, list(range(6)), use_unk_distribution=False, use_bert=False)
exp_num = explainer_num.explain_instance(text, predict_numerical, threshold=0.95)

In [ ]:
predicted_class_ids = np.argsort(pipeline.predict_proba([text])[0])[::-1]
pred = explainer_num.class_names[predicted_class_ids[0]]
alternative = explainer_num.class_names[predicted_class_ids[1]]
print(f'predicted {pred}, alternative {alternative}')

In [ ]:
exp_num.show_in_notebook()